In [6]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
from matplotlib import rcParams
import os


In [15]:
import pandas as pd
data=pd.read_csv('zomato.csv')


,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [19]:
#there are duplicate restaurants in the data that need to be removed
data[data.name == 'Addhuri Udupi Bhojana']

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
661,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Dine-out,Banashankari


In [9]:
#sort by name and address
data['name_address'] = data['name'] + data['address']

data.sort_values('name_address', inplace = True) 
  
# dropping ALL duplicte values 
data.drop_duplicates(subset ='name_address', 
                     keep = 'first', inplace = True) 


In [10]:
#parse review rows and split into list
all_ratings = []

for name,ratings in tqdm(zip(data['name'],data['reviews_list'])):
    ratings = eval(ratings)
    for score, doc in ratings:
        if score:
            score = score.strip("Rated").strip()
            doc = doc.strip('RATED').strip()
            score = float(score)
            all_ratings.append([name,score, doc])

12499it [00:02, 5201.21it/s]


In [23]:
#construct data frame
rating_df=pd.DataFrame(all_ratings,columns=['name','rating','review'])
rating_df['review']=rating_df['review'].apply(lambda x : re.sub('[^a-zA-Z0-9\s]',"",x))

#Remove blank reviews
rating_df = rating_df[rating_df['review'].apply(lambda x: x.strip()) != ""]

#remove duplicate reviews
rating_df['name_rating_review'] = rating_df['name'] + rating_df['rating'].apply(str) + rating_df['review']

rating_df.sort_values('name_rating_review', inplace = True) 


In [22]:
#After parsing the reviews we find more duplicates.

rating_df[rating_df.name == 'Addhuri Udupi Bhojana']

,name,rating,review,name_rating_review
5536,Addhuri Udupi Bhojana,1.5,The food was not satisfactory Not one item ser...,Addhuri Udupi Bhojana1.5The food was not satis...
5528,Addhuri Udupi Bhojana,2.0,How can a Udupi restaurant be so dirtythe floo...,Addhuri Udupi Bhojana2.0How can a Udupi restau...
5538,Addhuri Udupi Bhojana,2.0,Im rating 2 only because of the large number o...,Addhuri Udupi Bhojana2.0Im rating 2 only becau...
5526,Addhuri Udupi Bhojana,2.0,Reached the place at 3pm on Saturday Half of t...,Addhuri Udupi Bhojana2.0Reached the place at 3...
5549,Addhuri Udupi Bhojana,3.0,Been here last week with high expectations bas...,Addhuri Udupi Bhojana3.0Been here last week wi...
5540,Addhuri Udupi Bhojana,3.0,Been here last week with high expectations bas...,Addhuri Udupi Bhojana3.0Been here last week wi...
5546,Addhuri Udupi Bhojana,3.0,Been here last week with high expectations bas...,Addhuri Udupi Bhojana3.0Been here last week wi...
5543,Addhuri Udupi Bhojana,3.0,Been here last week with high expectations bas...,Addhuri Udupi Bhojana3.0Been here last week wi...
5552,Addhuri Udupi Bhojana,3.0,Been here last week with high expectations bas...,Addhuri Udupi Bhojana3.0Been here last week wi...
5559,Addhuri Udupi Bhojana,3.5,Aaaa wt to tell Unlimited food food was like e...,Addhuri Udupi Bhojana3.5Aaaa wt to tell Unlimi...


In [11]:
# dropping ALL duplicte values 
rating_df.drop_duplicates(subset ='name_rating_review', 
                     keep = 'first', inplace = True) 


In [13]:
rating_df

,name,rating,review,name_rating_review
2,#FeelTheROLL,4.5,Very nice place complete value for money High...,#FeelTheROLL4.5Very nice place complete value ...
3,#FeelTheROLL,5.0,Had an amazing mouthwatering chicken rollWorth...,#FeelTheROLL5.0Had an amazing mouthwatering ch...
0,#FeelTheROLL,5.0,Had an egg chicken roll and a paneer roll Real...,#FeelTheROLL5.0Had an egg chicken roll and a p...
1,#FeelTheROLL,5.0,Not just the Roll but the filling tastes great...,#FeelTheROLL5.0Not just the Roll but the filli...
5,#L-81 Cafe,4.0,A cozy cafe near silk board just came here bef...,#L-81 Cafe4.0A cozy cafe near silk board just ...
...,...,...,...,...
256042,re:cess - Hilton Bangalore Embassy GolfLinks,5.0,Stopped over post a late night party at recess...,re:cess - Hilton Bangalore Embassy GolfLinks5....
255940,re:cess - Hilton Bangalore Embassy GolfLinks,5.0,This is one of the few places in Bangalore tha...,re:cess - Hilton Bangalore Embassy GolfLinks5....
255939,re:cess - Hilton Bangalore Embassy GolfLinks,5.0,Visited here some time back very nice ambiance...,re:cess - Hilton Bangalore Embassy GolfLinks5....
255944,re:cess - Hilton Bangalore Embassy GolfLinks,5.0,We had some amazing fingerlicking grub at rece...,re:cess - Hilton Bangalore Embassy GolfLinks5....
